In [43]:
from rdflib import Graph, Literal
from rdflib.namespace import RDF, RDFS, SKOS
import rdflib
import os
from SPARQLWrapper import SPARQLWrapper, JSON
import pandas as pd
sparql = SPARQLWrapper("http://dbpedia.org/sparql")
sparql.setReturnFormat(JSON)
sparql.setTimeout(20)
schema = rdflib.Namespace("http://schema.org/")
owl = rdflib.Namespace("http://www.w3.org/2002/07/owl#")

In [8]:
directory = r"../../OpenCS/ontology/core/00/"
g = Graph()
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        new_node = Graph()
        new_node.parse(f)
        g += new_node
        

In [9]:
g.triples((None, RDF.type, schema.Place))

<generator object Graph.triples at 0x000002BBE4D5BE20>

In [24]:
URIRefs = [s for s, p, o in g.triples((None, SKOS.closeMatch, None))]
len(URIRefs)

589

In [42]:
URIRefs[0]

rdflib.term.URIRef('https://w3id.org/ocs/ont/C1')

In [26]:
names = ["(dbr:" + o.split("/").pop() + ")" for s, p, o in g.triples((None, SKOS.closeMatch, None))]
len(names)

589

In [27]:
names

['(dbr:Computer_science)',
 '(dbr:Algorithm)',
 '(dbr:Embedded_system)',
 '(dbr:Reconfigurability)',
 '(dbr:REEM)',
 '(dbr:Crt0)',
 '(dbr:Knowledge_management)',
 '(dbr:Systems_simulation)',
 '(dbr:Cellular_automaton)',
 '(dbr:Digital_filter)',
 '(dbr:Interrogative)',
 '(dbr:Next-generation_network)',
 '(dbr:Logical_consequence)',
 '(dbr:Bookmarking)',
 '(dbr:Global_variable)',
 '(dbr:Functional_capacity_evaluation)',
 '(dbr:Data_mining)',
 '(dbr:Instance_selection)',
 '(dbr:Java_Interface_Definition_Language)',
 '(dbr:Williot_diagram)',
 '(dbr:ClinicalKey)',
 '(dbr:Pro*C)',
 '(dbr:ThoughtTreasure)',
 '(dbr:Oracle_BI_server)',
 '(dbr:Reproduction_speed)',
 '(dbr:Greedy_source)',
 '(dbr:Computer_engineering)',
 '(dbr:Continuous_test-driven_development)',
 '(dbr:Computer_architecture)',
 '(dbr:Limiter)',
 '(dbr:Broadcast_engineering)',
 '(dbr:Abstract_machine)',
 '(dbr:ILLIAC)',
 '(dbr:Crosswind)',
 '(dbr:Timestamp)',
 '(dbr:Speech_recognition)',
 '(dbr:Geometric_primitive)',
 '(dbr:Meth

In [15]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [39]:
language_labels = []
languages = []
bsl = "\n"
#needs to be divied into chunks due to limitations of the rdfslib query
query = f"""

PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX dbr: <http://dbpedia.org/resource/>

SELECT DISTINCT
	 ?uri ?label 
WHERE {{
    VALUES (?uri) {{{bsl.join(names[0:1])}}}
    ?uri rdfs:label ?label
}}
LIMIT 1000
"""
sparql.setQuery(query)

try:
    ret = sparql.queryAndConvert()

    for r in ret["results"]["bindings"]:
        language_labels.append(r['label']['value'])
        languages.append(r['label']['xml:lang'])
except Exception as e:
    print(e)

In [45]:
for i in range(len(language_labels)):
    URIRef = URIRefs[0]
    lang_label = Literal(language_labels[i], lang = languages[i])
    g.add((URIRef, RDFS.label, lang_label))

In [46]:
[o for s, p, o in g.triples((None, RDFS.label, None))]

[rdflib.term.Literal('Computer science', lang='en'),
 rdflib.term.Literal('علم الحاسوب', lang='ar'),
 rdflib.term.Literal('Ciències de la computació', lang='ca'),
 rdflib.term.Literal('Matematická informatika', lang='cs'),
 rdflib.term.Literal('Informatik', lang='de'),
 rdflib.term.Literal('Επιστήμη υπολογιστών', lang='el'),
 rdflib.term.Literal('Informadiko', lang='eo'),
 rdflib.term.Literal('Konputazioaren zientziak', lang='eu'),
 rdflib.term.Literal('Ciencias de la computación', lang='es'),
 rdflib.term.Literal('Ríomheolaíocht', lang='ga'),
 rdflib.term.Literal('Ilmu komputer', lang='in'),
 rdflib.term.Literal('Informatique', lang='fr'),
 rdflib.term.Literal('Informatica', lang='it'),
 rdflib.term.Literal('컴퓨터 과학', lang='ko'),
 rdflib.term.Literal('計算機科学', lang='ja'),
 rdflib.term.Literal('Informatica', lang='nl'),
 rdflib.term.Literal('Informatyka', lang='pl'),
 rdflib.term.Literal('Ciência da computação', lang='pt'),
 rdflib.term.Literal('Информатика', lang='ru'),
 rdflib.term.Lit

In [50]:
def add_language_labels_from_dbpedia(filename=None, directory = r"../../OpenCS/ontology/core/00/"):
    f = os.path.join(directory, filename)
    g = Graph()
    if os.path.isfile(f):
        new_node = Graph()
        new_node.parse(f)
        g += new_node
    URIRefs = [s for s, p, o in g.triples((None, SKOS.closeMatch, None))]
    names = ["(dbr:" + o.split("/").pop() + ")" for s, p, o in g.triples((None, SKOS.closeMatch, None))]
    language_labels = []
    languages = []
    bsl = "\n"
    query = f"""

    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX dbr: <http://dbpedia.org/resource/>

    SELECT DISTINCT
        ?uri ?label 
    WHERE {{
        VALUES (?uri) {{{bsl.join(names[0:1])}}}
        ?uri rdfs:label ?label
    }}
    LIMIT 1000
    """
    sparql.setQuery(query)

    try:
        ret = sparql.queryAndConvert()

        for r in ret["results"]["bindings"]:
            language_labels.append(r['label']['value'])
            languages.append(r['label']['xml:lang'])
    except Exception as e:
        print(e)

    for i in range(len(language_labels)):
        lang_label = Literal(language_labels[i], lang = languages[i])
        g.add((URIRefs[0], RDFS.label, lang_label))

    g.serialize(destination=f)

In [53]:
add_language_labels_from_dbpedia(filename = r"C1.ttl")